# OIB to Ome-Zarr (Linescan Confocal Microscopy) Batch Conversion

This script performs batch conversion of all `.oib` files in a folder to the Ome-Zarr format.


## Define directory containing the files to be converted

In [1]:
data_path = '/home/jovyan/LNMA/guerreroa/data'
data_folder = '20241119'

## Install and load required libraries

In [2]:
print("Installing necessary libraries...")
!pip install natsort oiffile ome-zarr > /dev/null 2>&1
print("Libraries installed successfully.")

Installing necessary libraries...
Libraries installed successfully.


## Import Libraries

In [3]:
import oiffile
import os
import zarr
from ome_zarr.io import parse_url
from natsort import natsorted
import numpy as np

## Functions

In [4]:
def clean_filenames(directory_path):
    """
    Cleans filenames in the specified directory by replacing certain characters
    and renaming the files in the filesystem.

    Parameters:
        directory_path (str): Path to the directory containing the files.

    Returns:
        list: A list of cleaned file names sorted naturally.
    """
    # Initialize a list to store cleaned file names
    cleaned_file_names = []

    # Process each file in the specified directory
    for file_name in os.listdir(directory_path):
        # Construct the original full path
        original_path = os.path.join(directory_path, file_name)

        # Clean the filename by replacing spaces and other unwanted characters
        cleaned_name = file_name.replace(' ', '_').replace('-', '_').replace('/', '_')
        cleaned_name = cleaned_name.replace('+', '_').replace('_copy', '')

        # Construct the new full path
        cleaned_path = os.path.join(directory_path, cleaned_name)

        # Rename the file if the cleaned name is different from the original name
        if original_path != cleaned_path:
            os.rename(original_path, cleaned_path)
            #print(f"Renamed {original_path} to {cleaned_path}")

        # Append the cleaned file name to the list
        cleaned_file_names.append(cleaned_name)

    # Return the list of cleaned file names, sorted naturally
    return natsorted(cleaned_file_names)

## Clean file names

In [5]:
directory_path = os.path.join(data_path, data_folder)
cleaned_files = clean_filenames(directory_path)

# Exclude directories from the list
cleaned_files = [f for f in cleaned_files if not os.path.isdir(os.path.join(directory_path, f))]
cleaned_files


['20241119_CTRLXT_ConFluoBG_Ev_Ind_1000rep_C1.oib',
 '20241119_CTRLXT_ConFluoBG_Ev_Ind_2000rep_C1.oib',
 '20241119_CTRLXYT_ConFluoBG_cabezaflagelo_C1.oib',
 '20241119_XYT_CTRLMBF30uM_Frame200_cabezaflagelo_C1.oib']

In [6]:
directory_path

'/home/jovyan/LNMA/guerreroa/data/20241119'

## Create OME-Zarr files with metadata and rois included

In [7]:
for oib_file in cleaned_files:
    oib_file_path = os.path.join(directory_path, oib_file)

    # Skip directories
    if os.path.isdir(oib_file_path):
        print(f"Skipping directory: {oib_file_path}")
        continue

    # Skip files that do not have .oib extension
    if not oib_file.endswith('.oib'):
        print(f"Skipping non-oib file: {oib_file_path}")
        continue

    ome_zarr_path = f'{oib_file_path[:-4]}.zarr'
    print(f"Processing file: {oib_file_path}")

    try:
        with oiffile.OifFile(oib_file_path) as oib:
            image_data = oib.asarray()  # Load image data
            all_metadata = dict(oib.mainfile)  # Retrieve all metadata as a dictionary
        
        # Ensure data shape is in (T, C, Z, Y, X) format
        while image_data.ndim < 5:
            image_data = np.expand_dims(image_data, axis=0)  # Add dimensions as needed
        image_data = np.asarray(image_data).transpose(0, 2, 1, 3, 4)  # Reorder dimensions
        
        # Remove existing Zarr store if exists
        if os.path.exists(ome_zarr_path):
            import shutil
            shutil.rmtree(ome_zarr_path)
        
        # Save data and metadata to OME-Zarr
        store = parse_url(ome_zarr_path, mode="w").store
        root = zarr.group(store=store)
        root.create_dataset("image_data", data=image_data, chunks=True)
        root.attrs["image_metadata"] = all_metadata

        print(f"Successfully converted {oib_file_path} to {ome_zarr_path}")
    except Exception as e:
        print(f"Error processing {oib_file_path}: {e}")

print("Conversion complete. The OME-Zarr files are saved with all metadata.")


Processing file: /home/jovyan/LNMA/guerreroa/data/20241119/20241119_CTRLXT_ConFluoBG_Ev_Ind_1000rep_C1.oib
Successfully converted /home/jovyan/LNMA/guerreroa/data/20241119/20241119_CTRLXT_ConFluoBG_Ev_Ind_1000rep_C1.oib to /home/jovyan/LNMA/guerreroa/data/20241119/20241119_CTRLXT_ConFluoBG_Ev_Ind_1000rep_C1.zarr
Processing file: /home/jovyan/LNMA/guerreroa/data/20241119/20241119_CTRLXT_ConFluoBG_Ev_Ind_2000rep_C1.oib
Successfully converted /home/jovyan/LNMA/guerreroa/data/20241119/20241119_CTRLXT_ConFluoBG_Ev_Ind_2000rep_C1.oib to /home/jovyan/LNMA/guerreroa/data/20241119/20241119_CTRLXT_ConFluoBG_Ev_Ind_2000rep_C1.zarr
Processing file: /home/jovyan/LNMA/guerreroa/data/20241119/20241119_CTRLXYT_ConFluoBG_cabezaflagelo_C1.oib
Successfully converted /home/jovyan/LNMA/guerreroa/data/20241119/20241119_CTRLXYT_ConFluoBG_cabezaflagelo_C1.oib to /home/jovyan/LNMA/guerreroa/data/20241119/20241119_CTRLXYT_ConFluoBG_cabezaflagelo_C1.zarr
Processing file: /home/jovyan/LNMA/guerreroa/data/20241119